nikita

In [1]:
import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd
import seaborn as sns
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
%matplotlib notebook
from sklearn.linear_model import LogisticRegression
import sklearn.model_selection
import sklearn.preprocessing as preproc
from sklearn.feature_extraction import text
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
import pickle
import warnings
import gdown
warnings.filterwarnings("ignore")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [2]:
url = "https://drive.google.com/file/d/1wF_i-hjgcy3DIKzudYDd3FPsGm8HDDXx/view?usp=sharing"
file_id = url.split("/")[-2]
download_url = f"https://drive.google.com/uc?id={file_id}"
output_file = "dataset.csv"
gdown.download(download_url, output_file, quiet=False)
df = pd.read_csv(output_file)

Downloading...
From: https://drive.google.com/uc?id=14KMSPZZGQ25TDcYSLW722dKHmOOl-ObB
To: /content/dataset.csv
100%|██████████| 54.3M/54.3M [00:00<00:00, 165MB/s]


In [3]:
df.shape

(100000, 18)

In [4]:
training_data, test_data = sklearn.model_selection.train_test_split(df, test_size = 0.2, random_state=42)

bow_transform = CountVectorizer(tokenizer=lambda doc: doc, ngram_range=(3,3), lowercase=False)

X_tr_bow = bow_transform.fit_transform(training_data['comment_text'])
X_te_bow = bow_transform.transform(test_data['comment_text'])
y_tr = training_data['target']
y_te = test_data['target']

tfidf_transform = text.TfidfTransformer(norm=None)

X_tr_tfidf = tfidf_transform.fit_transform(X_tr_bow)
X_te_tfidf = tfidf_transform.transform(X_te_bow)

In [5]:
y_tr = (y_tr > 0).astype(int)
y_te = (y_te > 0).astype(int)

In [6]:
# from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, accuracy_score
# from sklearn.metrics import accuracy_score, classification_report

# def simple_logistic_classify(X_tr, y_tr, X_test, y_test, description, _C=1.0):
#     model = LogisticRegression().fit(X_tr, y_tr)
#     y_pred = model.predict(X_test)
#     score = accuracy_score(y_test, y_pred)

#     accuracy_tfidf = accuracy_score(y_test, y_pred)
#     print("Accuracy (TF-IDF):", accuracy_tfidf)
#     print(classification_report(y_test, y_pred))
#     return model

# model_bow = simple_logistic_classify(X_tr_bow, y_tr, X_te_bow, y_te, 'BOW')
# model_tfidf = simple_logistic_classify(X_tr_tfidf, y_tr, X_te_tfidf, y_te, 'TF-IDF')

Accuracy (TF-IDF): 0.81585
              precision    recall  f1-score   support

           0       0.75      0.77      0.76      7653
           1       0.86      0.84      0.85     12347

    accuracy                           0.82     20000
   macro avg       0.80      0.81      0.81     20000
weighted avg       0.82      0.82      0.82     20000

Accuracy (TF-IDF): 0.8276
              precision    recall  f1-score   support

           0       0.76      0.80      0.78      7653
           1       0.87      0.84      0.86     12347

    accuracy                           0.83     20000
   macro avg       0.82      0.82      0.82     20000
weighted avg       0.83      0.83      0.83     20000



In [7]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

def simple_logistic_classify(X_tr, y_tr, X_test, y_test, description, _C=1.0):
    # Define the logistic regression model
    model = LogisticRegression(C=_C)

    # Define the parameter grid for grid search
    param_grid = {'C': [0.01, 0.1, 1]}

    # Create the GridSearchCV object
    grid_search = GridSearchCV(model, param_grid, cv=3, scoring='accuracy')

    # Fit the grid search to the data
    grid_search.fit(X_tr, y_tr)

    # Get the best model from the grid search
    best_model = grid_search.best_estimator_

    # Predict on the test set using the best model
    y_pred = best_model.predict(X_test)

    # Print the accuracy and classification report
    accuracy_tfidf = accuracy_score(y_test, y_pred)
    print("Best hyperparameter (C) found:", grid_search.best_params_)
    print("Accuracy :", accuracy_tfidf)
    print(classification_report(y_test, y_pred))

    return best_model

# Apply grid search for finding the best C for BOW
model_bow = simple_logistic_classify(X_tr_bow, y_tr, X_te_bow, y_te, 'BOW')

# Apply grid search for finding the best C for TF-IDF
model_tfidf = simple_logistic_classify(X_tr_tfidf, y_tr, X_te_tfidf, y_te, 'TF-IDF')


Best hyperparameter (C) found: {'C': 1}
Accuracy (TF-IDF): 0.81585
              precision    recall  f1-score   support

           0       0.75      0.77      0.76      7653
           1       0.86      0.84      0.85     12347

    accuracy                           0.82     20000
   macro avg       0.80      0.81      0.81     20000
weighted avg       0.82      0.82      0.82     20000

Best hyperparameter (C) found: {'C': 0.01}
Accuracy (TF-IDF): 0.82985
              precision    recall  f1-score   support

           0       0.77      0.80      0.78      7653
           1       0.87      0.85      0.86     12347

    accuracy                           0.83     20000
   macro avg       0.82      0.82      0.82     20000
weighted avg       0.83      0.83      0.83     20000

